<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [3]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.20.2
Pandas:  1.3.0


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [3]:
# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 0)
print(bikes.head())

FileNotFoundError: [Errno 2] No such file or directory: 'dat/bikeshare.csv'

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [5]:
#ANSWER:
bikes = pd.read_table('bikeshare.csv', header = 0, delimiter = ",")
print(bikes.head())
print()

              datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2  2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3  2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4  2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  



Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [6]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [7]:
df = pd.read_excel('https://web.stanford.edu/~ashishg/msande111/excel/iris.xls', sheet_name = 'Sheet1')
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,Sepal Length (cm),Sepal Width (cm),Petal Length (cm),Petal Width (cm),Class,NaN,NaN,alpha,obj,NaN,NaN
1,NaN,7,3.2,4.7,1.4,Iris-versicolor,NaN,0.0,0,0,0.0,1.0
2,NaN,6.4,3.2,4.5,1.5,Iris-versicolor,NaN,0.0,NaN,NaN,0.0,1.0
3,NaN,6.9,3.1,4.9,1.5,Iris-versicolor,NaN,0.0,NaN,NaN,0.0,1.0
4,NaN,5.5,2.3,4,1.3,Iris-versicolor,NaN,0.0,NaN,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
96,NaN,4.8,3,1.4,0.3,Iris-setosa,NaN,NaN,NaN,NaN,0.0,1.0
97,NaN,5.1,3.8,1.6,0.2,Iris-setosa,NaN,NaN,NaN,NaN,0.0,1.0
98,NaN,4.6,3.2,1.4,0.2,Iris-setosa,NaN,NaN,NaN,NaN,0.0,1.0
99,NaN,5.3,3.7,1.5,0.2,Iris-setosa,NaN,NaN,NaN,NaN,0.0,1.0


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [48]:
#ANSWER
iris = pd.read_excel('https://web.stanford.edu/~ashishg/msande111/excel/iris.xls', sheet_name = 'Sheet1', header = 1,usecols = "B:F",nrows = 51)
iris

,Sepal Length (cm),Sepal Width (cm),Petal Length (cm),Petal Width (cm),Class
0,7.0,3.2,4.7,1.4,Iris-versicolor
1,6.4,3.2,4.5,1.5,Iris-versicolor
2,6.9,3.1,4.9,1.5,Iris-versicolor
3,5.5,2.3,4.0,1.3,Iris-versicolor
4,6.5,2.8,4.6,1.5,Iris-versicolor
5,5.7,2.8,4.5,1.3,Iris-versicolor
6,6.3,3.3,4.7,1.6,Iris-versicolor
7,4.9,2.4,3.3,1.0,Iris-versicolor
8,6.6,2.9,4.6,1.3,Iris-versicolor
9,5.2,2.7,3.9,1.4,Iris-versicolor


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [9]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2021-07-17T01:16:17.228Z,63.694600,-149.617000,113.60,2.50,ml,NaN,NaN,NaN,0.63,...,2021-07-17T01:24:34.292Z,"35 km W of McKinley Park, Alaska",earthquake,NaN,0.50,NaN,NaN,automatic,ak,ak
1,2021-07-17T00:59:52.702Z,-21.728800,-68.557100,148.35,4.70,mb,NaN,54.0,1.10300,0.54,...,2021-07-17T01:17:40.040Z,"89 km NNE of Calama, Chile",earthquake,6.20,6.60,0.138,16.0,reviewed,us,us
2,2021-07-17T00:53:25.018Z,-56.125700,-27.850300,94.71,5.60,mww,NaN,46.0,5.28000,0.91,...,2021-07-17T01:12:01.675Z,South Sandwich Islands region,earthquake,8.80,6.20,0.089,12.0,reviewed,us,us
3,2021-07-17T00:45:35.090Z,44.460667,-110.347167,6.34,3.63,ml,21.0,99.0,0.07655,0.21,...,2021-07-17T01:24:06.860Z,"63 km SSE of Mammoth, Wyoming",earthquake,0.47,1.18,0.218,18.0,reviewed,uu,uu
4,2021-07-17T00:44:54.100Z,38.539833,-119.453667,-0.49,3.46,ml,18.0,72.0,0.03750,0.09,...,2021-07-17T01:22:02.802Z,"29km SSW of Smith Valley, NV",earthquake,0.22,0.60,0.412,25.0,automatic,nc,nc


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [10]:
import pandas as pd

url = 'https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/'

df = pd.read_html(url)
df

[                         Bank NameBank           CityCity StateSt  CertCert  \
 0                    Almena State Bank             Almena      KS     15426   
 1           First City Bank of Florida  Fort Walton Beach      FL     16748   
 2                 The First State Bank      Barboursville      WV     14361   
 3                   Ericson State Bank            Ericson      NE     18265   
 4     City National Bank of New Jersey             Newark      NJ     21111   
 ..                                 ...                ...     ...       ...   
 558                 Superior Bank, FSB           Hinsdale      IL     32646   
 559                Malta National Bank              Malta      OH      6629   
 560    First Alliance Bank & Trust Co.         Manchester      NH     34264   
 561  National State Bank of Metropolis         Metropolis      IL      3815   
 562                   Bank of Honolulu           Honolulu      HI     21029   
 
                  Acquiring Institutio

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [4]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [13]:
#ANSWER
print(bikes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB
None


What are the column names in `bikes`?

In [21]:
#ANSWER
print(bikes.head(5))


              datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2  2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3  2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4  2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  
datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64

What are the data types of these columns?

In [22]:
#ANSWER
print (bikes.dtypes)

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object


What is the (row) index for this DataFrame?

In [23]:
#ANSWER
print (bikes.index)

RangeIndex(start=0, stop=10886, step=1)


https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [28]:
#ANSWER
print(bikes['season'])
print(bikes[['season']])

0        1
1        1
2        1
3        1
4        1
        ..
10881    4
10882    4
10883    4
10884    4
10885    4
Name: season, Length: 10886, dtype: int64
       season
0           1
1           1
2           1
3           1
4           1
...       ...
10881       4
10882       4
10883       4
10884       4
10885       4

[10886 rows x 1 columns]


How would we use object notation to show the first 4 rows of `atemp`?

In [30]:
#ANSWER
bikes['atemp'].head(4)

0    14.395
1    13.635
2    13.635
3    14.395
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [34]:
#E: bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
 bikes[[0,0]]

IndentationError: unexpected indent (Temp/ipykernel_13472/2341678324.py, line 4)

What is the correct way to access the 1st row of the DataFrame by its index?

In [36]:
#ANSWER
bikes.iloc[0]

datetime      2011-01-01 00:00:00
season                          1
holiday                         0
workingday                      0
weather                         1
temp                         9.84
atemp                      14.395
humidity                       81
windspeed                     0.0
casual                          3
registered                     13
count                          16
Name: 0, dtype: object

What is the correct way to access the 2nd column of the DataFrame by its index?

In [37]:
#ANSWER
bikes.iloc[1]

datetime      2011-01-01 01:00:00
season                          1
holiday                         0
workingday                      0
weather                         1
temp                         9.02
atemp                      13.635
humidity                       80
windspeed                     0.0
casual                          8
registered                     32
count                          40
Name: 1, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

?ANSWER: Detect missing values for an array-like object

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [38]:
bikes.isnull().head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [136]:
#ANSWER
bikes.isnull().sum()

datetime       0
season         0
holiday        0
workingday     0
weather        0
temp           0
atemp          0
humidity       0
windspeed      0
casual         0
registered     0
count          0
atemp_level    0
dtype: int64

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [45]:
#ANSWER
bikes.isnull().values.sum()

0

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [62]:
#ANSWER
def div(int1,int2):
    try: 
        answer = int1/int2
        return answer
    except:
        print('DOes not support these data types')
print(div(1,2))

0.5


Apply the Pandas `isna` function to the following data objects:

In [72]:
x = 2.3
y = np.nan
#print(y.dtype)
#print(x, y)
#dir(y)
type(y)

float

In [73]:
#ANSWER
x.isna()
#y.isna()

AttributeError: 'float' object has no attribute 'isna'

In [74]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [75]:
#ANSWER
na_values

AttributeError: 'numpy.ndarray' object has no attribute 'na_values'

How is the pandas I/O parameter `na_values` used?

? ANSWER: Is used to tell the pandas to consider any type of value and assign NaN  

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [85]:
#ANSWER
bikes['season'].value_counts()

4    2734
2    2733
3    2733
1    2686
Name: season, dtype: int64

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [94]:
#ANSWER
bikes.head()
bikes['windspeed'].values

array([ 0.    ,  0.    ,  0.    , ..., 15.0013,  6.0032,  8.9981])

Compute and print the overall minimum and maximum of the numeric data columns:

In [109]:
bikes_min, bikes_max = (bikes['temp'].min, bikes['temp'].max)
bikes_min, bikes_max

(<bound method NDFrame._add_numeric_operations.<locals>.min of 0         9.84
 1         9.02
 2         9.02
 3         9.84
 4         9.84
          ...  
 10881    15.58
 10882    14.76
 10883    13.94
 10884    13.94
 10885    13.12
 Name: temp, Length: 10886, dtype: float64>,
 <bound method NDFrame._add_numeric_operations.<locals>.max of 0         9.84
 1         9.02
 2         9.02
 3         9.84
 4         9.84
          ...  
 10881    15.58
 10882    14.76
 10883    13.94
 10884    13.94
 10885    13.12
 Name: temp, Length: 10886, dtype: float64>)

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [99]:
bikes['atemp'].quantile(0.5)

24.24

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [102]:
#ANSWER
bikes['atemp'].quantile((0.1,0.25,0.5,0.75,0.9))

0.10    12.120
0.25    16.665
0.50    24.240
0.75    31.060
0.90    34.090
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [103]:
type(bikes['atemp'])

pandas.core.series.Series

In [104]:
bikes.sample(5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
7282,2012-05-02 19:00:00,2,0,1,1,22.14,25.760,77,16.9979,43,461,504
1459,2011-04-06 16:00:00,2,0,1,1,22.14,25.760,28,27.9993,27,142,169
1260,2011-03-17 08:00:00,1,0,1,1,14.76,17.425,66,8.9981,17,282,299
5107,2011-12-06 21:00:00,4,0,1,2,18.04,21.970,100,12.9980,6,108,114
8493,2012-07-15 06:00:00,3,0,0,1,27.88,31.820,83,8.9981,6,9,15


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [110]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [111]:
#ANSWER
atemp_level

0        (11.934, 23.108]
1        (11.934, 23.108]
2        (11.934, 23.108]
3        (11.934, 23.108]
4        (11.934, 23.108]
               ...       
10881    (11.934, 23.108]
10882    (11.934, 23.108]
10883    (11.934, 23.108]
10884    (11.934, 23.108]
10885    (11.934, 23.108]
Name: atemp, Length: 10886, dtype: category
Categories (4, interval[float64, right]): [(0.715, 11.934] < (11.934, 23.108] < (23.108, 34.281] < (34.281, 45.455]]

Here is a random sample of `atemp_level`:

In [112]:
atemp_level.sample(5)          

4541     (11.934, 23.108]
6667     (23.108, 34.281]
3407     (23.108, 34.281]
8982     (34.281, 45.455]
10252    (23.108, 34.281]
Name: atemp, dtype: category
Categories (4, interval[float64, right]): [(0.715, 11.934] < (11.934, 23.108] < (23.108, 34.281] < (34.281, 45.455]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [113]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

8805    warm
4704    mild
763     cool
2678    warm
5314    mild
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [135]:
#ANSWER
bikes['atemp_level'] = atemp_level
#bikes
bikes.loc[(bikes['season']==2) & (bikes['atemp_level']=="mild")].count


<bound method DataFrame.count of                  datetime  season  holiday  workingday  weather   temp  \
1323  2011-04-01 00:00:00       2        0           1        3  10.66   
1324  2011-04-01 01:00:00       2        0           1        3  10.66   
1325  2011-04-01 02:00:00       2        0           1        3  10.66   
1330  2011-04-01 07:00:00       2        0           1        3   9.84   
1331  2011-04-01 08:00:00       2        0           1        2  10.66   
...                   ...     ...      ...         ...      ...    ...   
7749  2012-06-03 06:00:00       2        0           0        1  18.86   
7818  2012-06-06 03:00:00       2        0           1        1  18.86   
7819  2012-06-06 04:00:00       2        0           1        1  18.86   
7843  2012-06-07 04:00:00       2        0           1        1  18.86   
7845  2012-06-07 06:00:00       2        0           1        1  18.86   

       atemp  humidity  windspeed  casual  registered  count atemp_level  
132

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [151]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )
print (a.shape, a)
print (b.shape, b)
print (c.shape, c)
print (d.shape, d)
print (e.shape, e)

(3,) [0. 0. 0.]
(1, 3) [[1. 1. 1.]]
(2, 3, 4) [[[3 8 2 1]
  [5 4 9 3]
  [1 1 6 6]]

 [[2 3 3 9]
  [1 6 6 7]
  [6 5 1 9]]]
(4,) [0 1 2 3]
(2, 4) [[1 2 3 4]
 [5 6 7 8]]


In [217]:
# Cleaning Data
rock




,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2224,She Loves My Automobile,ZZ Top,0,She Loves My Automobile by ZZ Top,1,0,1,0
2225,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2226,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2227,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


In [ ]:
#rock['Release Year']=='SONGFACTS.COM'
rock[rock['Release Year']=='SONGFACTS.COM'] = 1984
rock['Release Year']= rock['Release Year'].astype(int)
rock.dtypes

## Load Data

Load rock.csv and clean the dataset.

In [ ]:
rock = pd.read_csv("classic-rock-song-list.csv")

## Check Column Names

Check column names and clean.

In [ ]:
print(rock.head())

## Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [187]:
rock['Release Year'].fillna(0,inplace=True)

## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [191]:
rock[rock['Release Year']=='SONGFACTS.COM'] = 1984
rock['Release Year']= rock['Release Year'].astype(int)
rock.dtypes

Song Clean      object
ARTIST CLEAN    object
Release Year     int32
COMBINED        object
First?           int64
Year?            int64
PlayCount        int64
F*G              int64
dtype: object

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [216]:
print(rock['Release Year'].max(),rock['Release Year'].min())
print(rock['First?'].max(),rock['First?'].min())
print(rock['Year?'].max(),rock['Year?'].min())
print(rock['PlayCount'].max(),rock['PlayCount'].min())
print(rock['F*G'].max(),rock['F*G'].min())

2014 0
1984 1
1984 0
1984 0
1984 0


## Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [220]:
def printinfo(row):
    if row['Release Year'] < 1970:
        year = "Before 1970"
    else:
        year = "After 1970"    
    print(row['Song Clean'],row['ARTIST CLEAN'], year)
#printinfo(rock.iloc[4])


Art For Arts Sake 10cc After 1970


## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [ ]:
rock2 = pd.read_csv("classic-rock-song-list.csv")
def convert(DF, row):
       try:
        DF['row'] = DF['row'].astype(float)
        except:
        DF['row']= np.nan  

## Apply these functions to your dataset

## Describe the new float-only DataFrame.

>




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



